# Tarea: tiempos de ejecución con distintas estrategias

Este cuaderno implementa 3 problemas (P1, P2, P3). Para cada uno hay 4 versiones:
- Bucle `for`
- List comprehension
- Generador (materializado con `list(...)`)
- NumPy vectorizado

Se verifica primero la corrección y luego se miden tiempos con `timeit` usando los mismos parámetros para todas las versiones.


## Setup y utilidades de medición
- Dependencias mínimas: `numpy`
- Función auxiliar `bench_callable` para medir tiempos de forma uniforme (mismo `repeat` y `number`).


In [ ]:
import numpy as np
import random
import timeit
from statistics import mean

random.seed(123)
np.random.seed(123)

def bench_callable(fn, number=1, repeat=3):
    t = timeit.Timer(fn)
    times = t.repeat(repeat=repeat, number=number)
    return min(times), mean(times)

def show_bench(title, items, number=1, repeat=3):
    print(title)
    for label, fn in items:
        best, avg = bench_callable(fn, number=number, repeat=repeat)
        print(f'  {label:>16}: best={best:.6f}s avg={avg:.6f}s')
    print()


## P1 — Cuadrados de 0..n-1
Objetivo: construir la lista de cuadrados `[i*i for i in range(n)]`.

Versiones: `for`, list comprehension, generador (materializado con `list(...)`) y NumPy vectorizado.


In [ ]:
# P1 — Implementaciones
def p1_for(n: int):
    out = []
    for i in range(n):
        out.append(i*i)
    return out

def p1_listcomp(n: int):
    return [i*i for i in range(n)]

def p1_generator(n: int):
    # Generador que produce los cuadrados; se materializa con list(...) al medir
    for i in range(n):
        yield i*i

def p1_numpy(n: int):
    return (np.arange(n) ** 2)


In [ ]:
# P1 — Verificación de corrección
n_test = 10
ref = p1_listcomp(n_test)
assert p1_for(n_test) == ref
assert list(p1_generator(n_test)) == ref
assert list(p1_numpy(n_test).tolist()) == ref
print('P1 verificación OK')


In [ ]:
# P1 — Benchmark
Ns = [50_000, 200_000, 500_000]
for n in Ns:
    show_bench(
        title=f'P1 tiempos para n={n}',
        items=[
            ('for',           lambda n=n: p1_for(n)),
            ('listcomp',      lambda n=n: p1_listcomp(n)),
            ('generator(list)', lambda n=n: list(p1_generator(n))),
            ('numpy',         lambda n=n: p1_numpy(n)),
        ],
        number=1,
        repeat=3,
    )


Comentario breve esperado para P1:
- `numpy` suele ser el más rápido (vectorización en C).
- `list comprehension` tiende a superar al `for` en Python puro.
- El generador materializado (`list(gen)`) suele estar cerca de `for`/`listcomp` según el patrón.


## P2 — Transformación elemento a elemento con polinomio
Dado un arreglo/lista de `n` flotantes en [0, 1), calcular `f(x) = 3x**2 + 2x + 1` para cada elemento.

Separamos preparación de datos de la medición para ser justos.


In [ ]:
# P2 — Implementaciones
def f_poly(x):
    return 3*x*x + 2*x + 1

def p2_for(xs_list):
    out = []
    for x in xs_list:
        out.append(f_poly(x))
    return out

def p2_listcomp(xs_list):
    return [f_poly(x) for x in xs_list]

def p2_generator(xs_list):
    for x in xs_list:
        yield f_poly(x)

def p2_numpy(xs_np):
    x = xs_np
    return 3*(x**2) + 2*x + 1


In [ ]:
# P2 — Verificación de corrección
n_test = 10
xs_list = [random.random() for _ in range(n_test)]
xs_np = np.array(xs_list, dtype=float)
ref = p2_listcomp(xs_list)
assert p2_for(xs_list) == ref
assert list(p2_generator(xs_list)) == ref
assert np.allclose(p2_numpy(xs_np), ref)
print('P2 verificación OK')


In [ ]:
# P2 — Benchmark
Ns = [100_000, 300_000, 500_000]
for n in Ns:
    xs_list = [random.random() for _ in range(n)]
    xs_np = np.array(xs_list, dtype=float)
    show_bench(
        title=f'P2 tiempos para n={n}',
        items=[
            ('for',              lambda xs_list=xs_list: p2_for(xs_list)),
            ('listcomp',         lambda xs_list=xs_list: p2_listcomp(xs_list)),
            ('generator(list)',  lambda xs_list=xs_list: list(p2_generator(xs_list))),
            ('numpy',            lambda xs_np=xs_np: p2_numpy(xs_np)),
        ],
        number=1,
        repeat=3,
    )


Comentario breve esperado para P2:
- NumPy mantiene ventaja clara en transformaciones vectorizadas.
- Entre Python puro, `list comprehension` y `for` suelen ser competitivos; el generador materializado queda similar.


## P3 — Clasificación de puntos en un círculo
Dados `n` puntos 2D (x, y) en [-1, 1), determinar si están dentro del círculo unitario: `x*x + y*y <= 1`.

Salida: lista/array booleana del mismo tamaño.


In [ ]:
# P3 — Implementaciones
def p3_for(pts_list):
    out = []
    for x, y in pts_list:
        out.append((x*x + y*y) <= 1.0)
    return out

def p3_listcomp(pts_list):
    return [(x*x + y*y) <= 1.0 for (x, y) in pts_list]

def p3_generator(pts_list):
    for x, y in pts_list:
        yield (x*x + y*y) <= 1.0

def p3_numpy(pts_np):
    # pts_np shape: (n, 2)
    x = pts_np[:, 0]
    y = pts_np[:, 1]
    return (x*x + y*y) <= 1.0


In [ ]:
# P3 — Verificación de corrección
n_test = 10
pts_list = [(random.uniform(-1, 1), random.uniform(-1, 1)) for _ in range(n_test)]
pts_np = np.array(pts_list, dtype=float)
ref = p3_listcomp(pts_list)
assert p3_for(pts_list) == ref
assert list(p3_generator(pts_list)) == ref
assert p3_numpy(pts_np).tolist() == ref
print('P3 verificación OK')


In [ ]:
# P3 — Benchmark
Ns = [100_000, 300_000, 600_000]
for n in Ns:
    pts_list = [(random.uniform(-1, 1), random.uniform(-1, 1)) for _ in range(n)]
    pts_np = np.array(pts_list, dtype=float)
    show_bench(
        title=f'P3 tiempos para n={n}',
        items=[
            ('for',              lambda pts_list=pts_list: p3_for(pts_list)),
            ('listcomp',         lambda pts_list=pts_list: p3_listcomp(pts_list)),
            ('generator(list)',  lambda pts_list=pts_list: list(p3_generator(pts_list))),
            ('numpy',            lambda pts_np=pts_np: p3_numpy(pts_np)),
        ],
        number=1,
        repeat=3,
    )


## Observaciones finales
- En general, NumPy vectorizado domina en rendimiento para operaciones elementales y aritmética de arreglos.
- Entre las alternativas de Python puro, `list comprehension` suele ser más rápido que el `for` explícito; materializar un generador con `list(...)` queda cercano.
- A medida que `n` crece, las ventajas de NumPy se vuelven más notorias.

Sugerencia: ajusta `Ns`, `repeat` y `number` si tu máquina es lenta o deseas mediciones más estables.
